In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%aimport AD_predictor_tools
%aimport PlottingTools
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
sns.set_theme()
import protfasta
warnings.filterwarnings('ignore')
from matplotlib.ticker import PercentFormatter
from scipy.stats import ks_2samp, ttest_ind
sns.set(style = "white", rc={'figure.figsize':(6.5,4)}, font_scale = 1.2)
import matplotlib.ticker as mtick
import math

Please do a t-test for each residue to test if average in ADs is greater than average in all TFs. I expect few or none to be significant

In [28]:
def return_pval_df(AD_list):
    AAs = ['A', 'I', 'L', 'M', 'V', 'F', 'W',
           'Y', 'N', 'C', 'Q', 'S', 'T', 'D', 'E', 'R', 'H', 'K', 'G', 'P']

    lambert_TFs = pd.read_csv("../data/LambertTFs.csv")
    lambert_TFs["Length"] = lambert_TFs["ProteinSeq"].str.len()
    AD_list["Length"] = AD_list["Sequence"].str.len()
    lambert_TFs

    for AA in AAs:
        lambert_TFs[AA + "_prop"] = lambert_TFs["ProteinSeq"].str.count(AA) / lambert_TFs["Length"]


    for AA in AAs:
        AD_list[AA + "_prop"] = AD_list["Sequence"].str.count(AA) / AD_list["Length"]

    stats = []
    pvals = []

    
    for AA in AAs:
        result = ttest_ind(AD_list[AA + "_prop"], 
            lambert_TFs[AA + "_prop"], axis=0, equal_var= False, nan_policy='propagate', alternative='greater')
        stats.append(result[0])
        pvals.append(result[1])

    pval_df = pd.DataFrame({"AA": AAs,
                 "stat": stats,
                 "pval": pvals})
    
    display(AD_list)
    return pval_df.sort_values("pval")

In [14]:
GSL = pd.read_csv("../data/newGSL.csv")
Soto = pd.read_csv("../data/SotoEtAl_ADs.csv")
activity_data = pd.read_csv("../data/Staller2021/SupplementalDataSet4_ActivityData_PredictedADs_renorm20210708.csv")

In [23]:
active = activity_data[activity_data["Activity_mean"] > 221]
active = active.rename(columns = {"ProteinRegionSeq" : "Sequence"})
active

,Unnamed: 0,GeneName,Sequence,Activity_mean,Activity_SEM,RegionType,Start,End,Length,OverlapsKRAB,...,NegativeControls,PositiveControls,Activity_Zscore_mean,Activity_Zscore_SEM,Biological_Replicate_1_Raw_Activity,Biological_Replicate_2_Raw_Activity,Biological_Replicate_3_Raw_Activity,Biological_Replicate_1_UniqueIntegrationBCs,Biological_Replicate_2_UniqueIntegrationBCs,Biological_Replicate_3_UniqueIntegrationBCs
0,0,>sp|Q8N587|ZN561_HUMAN,VEDYLASGYQDSVTFDDVAVDFTPEEWALLDTTEKYLYRD,223.334567,7.511416,Prediction,28.0,68.0,41.0,True,...,False,False,0.453366,0.108106,232.332770,208.417290,229.253643,63.0,69.0,77.0
5,5,>sp|Q8IWI9|MGAP_HUMAN,VTSAIEEAALDSSELLTNMEDEDDTDETLTSLLNEIAFLNQQLNDD...,255.847405,23.439272,Prediction,2794.0,2855.0,62.0,False,...,False,False,1.416183,0.699873,213.105172,260.544455,293.892588,4.0,5.0,6.0
15,15,>sp|P01100|FOS_HUMAN,DLGFPEEMSVASLDLTGGLPEVATPESEEAFTLPLLND,238.542529,18.484197,Prediction,208.0,246.0,39.0,False,...,False,False,0.852041,0.291092,227.016103,213.886131,274.725353,15.0,17.0,18.0
16,16,>sp|P78545|ELF3_HUMAN,DELSWIIELLEKDGMAFQEALDPGPFDQGSPFAQELLDDG,260.767706,30.305323,Prediction,133.0,173.0,41.0,False,...,False,False,1.468025,0.599111,275.470923,202.493613,304.338583,6.0,16.0,10.0
17,17,>sp|Q9H582|ZN644_HUMAN,SYETEDESSWDNVELGDYTTQAIEDETYSDINQEHVNLFPLF,235.175232,22.645855,Prediction,851.0,893.0,43.0,False,...,False,False,0.745198,0.487600,207.846517,217.560840,280.118340,3.0,4.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,429,DWs,DWDWDWDWDWDWDWDWDWDWDWDWD,234.821019,2.537864,Controls,0.0,1.0,25.0,False,...,False,True,0.821934,0.255981,231.855094,232.736808,239.871155,96.0,78.0,90.0
430,430,DFs,DFDFDFDFDFDFDFDFDFDFDFDFD,243.668151,1.963449,Controls,0.0,1.0,25.0,False,...,False,True,1.092590,0.278373,241.881601,241.532963,247.589887,19.0,29.0,20.0
432,432,VP16_F442A,PTDVSLGDELHLDGEDVAMAHADALDDADLDMLGDGDSP,232.560501,18.343757,Controls,0.0,1.0,39.0,False,...,True,False,NaN,NaN,210.094079,NaN,255.026922,2.0,1.0,3.0
433,433,HIF1a_AD2_46_RK>D,DLLGQSMDESGLPQLTSYDCEVNAPIQGSDNLLQGEELLDALDQVN,243.059417,14.360576,Controls,0.0,1.0,46.0,False,...,False,True,1.007849,0.123329,244.344329,217.568618,267.265305,12.0,11.0,9.0


In [30]:
GSL_pval_df = return_pval_df(GSL)
GSL_pval_df[GSL_pval_df["pval"] < (0.01 / 20)]

,Unnamed: 0,GeneName,AD name,Start,End,uniprotID,Reference,Sequence,Length,A_prop,...,Q_prop,S_prop,T_prop,D_prop,E_prop,R_prop,H_prop,K_prop,G_prop,P_prop
0,0,OTX2,Cterm,231,289,P32243,uniprot,LNQSPASLSTQGYGASSLGFNSTTDCLDYKDQTASWKLNFNADCLD...,59,0.067797,...,0.084746,0.152542,0.084746,0.101695,0.000000,0.000000,0.000000,0.067797,0.050847,0.016949
1,1,CRX,Cterm,200,284,O43186,Chen 2002 pg 873,AFCSSPSAYGSPSSYFSGLDPYLSPMVPQLGGPALSPLSGPSVGPS...,85,0.058824,...,0.035294,0.211765,0.047059,0.047059,0.011765,0.000000,0.000000,0.023529,0.105882,0.152941
2,2,VP16,Full / H1 / MVS N_C (acid blob library fragmen...,411,490,P06492,nan / nan / nan / nan,STAPPTDVSLGDELHLDGEDVAMAHADALDDFDLDMLGDGDSPGPG...,80,0.112500,...,0.012500,0.050000,0.050000,0.200000,0.062500,0.000000,0.037500,0.000000,0.125000,0.075000
3,3,Oct4,Nterm,1,137,Q01860,Brehm,MAGHLASDFAFSPPPGGGGDGPGGPEPGWVDPRTWLSFQGPPGGPG...,137,0.051095,...,0.043796,0.065693,0.029197,0.029197,0.094891,0.007299,0.007299,0.021898,0.218978,0.175182
4,4,Oct4,Cterm,290,360,Q01860,Boija 2018,SDYAQREDFEAAGSPFSGGPVSFPLAPGPHFGTPGYGSPHFTALYS...,71,0.084507,...,0.014085,0.140845,0.056338,0.028169,0.056338,0.014085,0.042254,0.000000,0.126761,0.169014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,163,AKAP8L,NaN,1,268,Q9ULX6,activation_regions.txt,MSYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYG...,268,0.059701,...,0.048507,0.111940,0.070896,0.063433,0.048507,0.078358,0.007463,0.007463,0.152985,0.037313
163,164,ESRRB,NaN,203,433,O95718,transcriptionalactivity_regions.txt,PPAKKPLTKIVSYLLVAEPDKLYAMPPPGMPEGDIKALTTLCDLAD...,231,0.082251,...,0.043290,0.051948,0.030303,0.056277,0.082251,0.038961,0.025974,0.082251,0.038961,0.056277
164,165,AHR,NaN,118,126,P35869,transcriptionalactivity_regions.txt,LLQALNGFV,9,0.111111,...,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000
165,166,AHR,NaN,266,268,P35869,transcriptionalactivity_regions.txt,FAI,3,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,AA,stat,pval
13,D,7.487247,1.844719e-12
3,M,4.537939,5.347516e-06
2,L,4.434833,8.194069e-06
11,S,4.149248,2.612350e-05
19,P,3.845096,8.437518e-05


In [25]:
Soto_pval_df = return_pval_df(Soto)
Soto_pval_df[Soto_pval_df["pval"] < (0.01 / 20)]

,AA,stat,pval
19,P,10.417761,7.132361e-24
13,D,9.606161,1.269694e-20
11,S,8.965050,2.127173e-18
3,M,7.723430,2.598165e-14
2,L,5.431463,4.111840e-08
7,Y,4.073530,2.648468e-05
0,A,3.452923,2.948681e-04


In [29]:
active_pval_df = return_pval_df(active)
active_pval_df#[active_pval_df["pval"] < (0.01 / 20)]

,Unnamed: 0,GeneName,Sequence,Activity_mean,Activity_SEM,RegionType,Start,End,Length,OverlapsKRAB,...,Q_prop,S_prop,T_prop,D_prop,E_prop,R_prop,H_prop,K_prop,G_prop,P_prop
0,0,>sp|Q8N587|ZN561_HUMAN,VEDYLASGYQDSVTFDDVAVDFTPEEWALLDTTEKYLYRD,223.334567,7.511416,Prediction,28.0,68.0,40,True,...,0.025000,0.050000,0.100000,0.175000,0.100000,0.025000,0.000000,0.025,0.025000,0.025000
5,5,>sp|Q8IWI9|MGAP_HUMAN,VTSAIEEAALDSSELLTNMEDEDDTDETLTSLLNEIAFLNQQLNDD...,255.847405,23.439272,Prediction,2794.0,2855.0,61,False,...,0.032787,0.114754,0.098361,0.131148,0.147541,0.000000,0.000000,0.000,0.016393,0.016393
15,15,>sp|P01100|FOS_HUMAN,DLGFPEEMSVASLDLTGGLPEVATPESEEAFTLPLLND,238.542529,18.484197,Prediction,208.0,246.0,38,False,...,0.000000,0.078947,0.078947,0.078947,0.157895,0.000000,0.000000,0.000,0.078947,0.105263
16,16,>sp|P78545|ELF3_HUMAN,DELSWIIELLEKDGMAFQEALDPGPFDQGSPFAQELLDDG,260.767706,30.305323,Prediction,133.0,173.0,40,False,...,0.075000,0.050000,0.000000,0.150000,0.125000,0.000000,0.000000,0.025,0.100000,0.075000
17,17,>sp|Q9H582|ZN644_HUMAN,SYETEDESSWDNVELGDYTTQAIEDETYSDINQEHVNLFPLF,235.175232,22.645855,Prediction,851.0,893.0,42,False,...,0.047619,0.095238,0.095238,0.119048,0.166667,0.000000,0.023810,0.000,0.023810,0.023810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,429,DWs,DWDWDWDWDWDWDWDWDWDWDWDWD,234.821019,2.537864,Controls,0.0,1.0,25,False,...,0.000000,0.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
430,430,DFs,DFDFDFDFDFDFDFDFDFDFDFDFD,243.668151,1.963449,Controls,0.0,1.0,25,False,...,0.000000,0.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
432,432,VP16_F442A,PTDVSLGDELHLDGEDVAMAHADALDDADLDMLGDGDSP,232.560501,18.343757,Controls,0.0,1.0,39,False,...,0.000000,0.051282,0.025641,0.282051,0.051282,0.000000,0.051282,0.000,0.102564,0.051282
433,433,HIF1a_AD2_46_RK>D,DLLGQSMDESGLPQLTSYDCEVNAPIQGSDNLLQGEELLDALDQVN,243.059417,14.360576,Controls,0.0,1.0,46,False,...,0.108696,0.086957,0.021739,0.130435,0.086957,0.000000,0.000000,0.000,0.086957,0.043478


,AA,stat,pval
2,L,9.518497,5.149437e-17
13,D,8.379880,3.684633e-14
14,E,5.818430,2.134766e-08
5,F,3.312280,5.980242e-04
3,M,3.237277,7.612011e-04
6,W,1.414677,7.977331e-02
19,P,0.712450,2.386891e-01
11,S,0.449019,3.270755e-01
8,N,-0.455025,6.750892e-01
10,Q,-1.166899,8.773383e-01


---
Please check if S rich ADs from soto (or GSL) are S-rich compared to TF regions

In [ ]:
Soto = pd.read_csv("../data/SotoEtAl_ADs.csv")
Soto

In [ ]:
Soto["S_prop"] = Soto["Sequence"].str.count("S") / (1 + Soto["End"] - Soto["Start"])
Soto

In [ ]:
S_rich_Soto = Soto[Soto["S_prop"] > 0.15]

In [ ]:
ttest_ind(S_rich_Soto["S" + "_prop"], 
        lambert_TFs["S" + "_prop"], axis=0, equal_var= False, nan_policy='propagate', alternative='greater')

---
Can you also do S+P together. Remind me to tell you the phosphorylation story that is brewing.

In [ ]:
Soto["SP_prop"] = (2* Soto["Sequence"].str.count("SP")) / (1 + Soto["End"] - Soto["Start"])
Soto

In [ ]:
lambert_TFs["SP_prop"] = (2 * lambert_TFs["ProteinSeq"].str.count("SP")) / (lambert_TFs["Length"])

In [ ]:
ttest_ind(Soto["SP" + "_prop"], 
        lambert_TFs["SP" + "_prop"], axis=0, equal_var= False, nan_policy='propagate', alternative='greater')